# Door codes

After a fairly dull flight, you've finally arrived at your hotel. The good news is that the hotel has high-security electronic locks on the room doors. The bad news is that the staff are rather busy, and you think it will take a long time to get to your room.

Luckily, you know how their system works. 

Each door in the hotel has a keyboard on the lock. You have to enter the correct two-letter code to get in to the room. Because the staff know that people won't remember the codes, they tell you a pass phrase you can use the generate the code from.

There's a long queue for people to be told how to generate the code from the pass phrase. You were here last year and you still remember how to do it.

You start with the first two letters of the pass phrase. This is the starting value of your code.

Then, for each subsequent letter in the pass phrase, you:
1. "Add" the second letter of the code to the first letter of the code, replacing the first letter of the code
2. "Add" the current letter of the pass phrase to the second letter of the code, replacing the second letter of the code
3. Move on to the next letter of the pass phrase

"Adding" letters is done by converting the letters to their position in the alphabet (starting at one), adding, then converting the number back to a letter. Numbers greater than 26 are "wrapped around". For instance, to add `t` + `h`, convert them to numbers (`t`=`20`, `h`=`8`), add them (`20` + `8` = `28`), then convert back to a letter (`28` is larger than 26, so it becomes `28` - `26` = `2`, which is `b`).

All letters are converted to lower-case, and anything that isn't a letter is ignored.

For example, to find the code from the pass phrase `the cat`, the code starts as being the first two letters `th`, then the subsequent letters are used to update the code as follows:

| old code | code as<br>numbers | passphrase<br/>letter | number of<br/>letter | new first<br/>part of code |new second<br/>part of code | new code |
|:---|:---|:---|:---|:---|:---|:---|
| `th` | (20, 8) | `e` | 5 | 28 | 13 | `bm` |
| `bm` | (2, 13) | `c` | 3 | 15 | 16 | `op` |
| `op` | (15, 16) | `a` | 1 | 31 | 17 | `eq` |
| `eq` | (5, 17) | `t` | 20 | 22 | 37 | `vk` |

So the final code is `vk`

## Part 1
Your passphrase is 
"the traveller in the grey riding-coat, who called himself mr. melville, was contemplating the malice of which the gods are capable."

**What is your door code?**

# Worked example of solution: Part 1

While the overall shape of this is the same as previous days (walk along a list, updating the code as you reach each letter), there are a couple of wrinkles:

1. Not every character in the input should be processed (and the others should be converted to lower-case letters).
2. The 'update the code' part is complex.

"Sanitising" the input is, again, walking over the input, convering letters and discarding the rest. These are examples of standard approaches: `filter` is applying a predicate to every item in a sequence, returning just hose that pass; `map` is applying a function to every item in a sequence, returning the sequence of results. In this case, sanitising the input is `filter`ing to keep just the letters then `map`ping over the "convert to lowercase" function. Python's comprehensions do this: the general form is `f(x) for x in sequence if predicate(x)`

Updating the code involves lots of faffing around, converting between characters and numbers. Rather than retyping lots of arithmetic, I define a couple of functions to do the conversions how I want. I've deliberately given them short names, as I want the functions to almost disappear in the program, becoming little more than punctuation. That will keep the focus on the important part, the updating.

The `ord(letter) - ord('a')` and `chr(number + ord('a')` are standard idioms for converting from letters to positions in the alphabet. There's also moving the result by 1 to give one-based numbering, and the modulus operation `%` to keep the numbers in the range 0-25 before converting back to letters.

Finally, the `string` library defines some convenient constants, which helps prevent annoying and hard-to-find typos if I wrote out the alphabet verbatim here.

In [2]:
import string

In [5]:
def sanitise(phrase):
    return ''.join(l.lower() for l in phrase if l in string.ascii_letters)

In [6]:
sanitise('Hello World')

'helloworld'

In [3]:
def o(letter):
    return ord(letter) - ord('a') + 1

def c(number):
    return chr((number - 1) % 26 + ord('a'))

In [4]:
c(1), c(0), c(26), c(27)

('a', 'z', 'z', 'a')

In [23]:
def whash1(word):
    h = list(word[:2])
    for l in word[2:]:
        h[0] = c(o(h[0]) + o(h[1]))
        h[1] = c(o(h[1]) + o(l))
    return ''.join(h)

In [20]:
# Extended version that generates the tables used in the question text.
def whash1(word, show_steps=False):
    if show_steps:
        print('| old code | code as<br>numbers | passphrase<br/>letter | number of<br/>letter | new first<br/>part of code |'
              'new second<br/>part of code | new code |')
        print('|:---|:---|:---|:---|:---|:---|:---|')
    h = list(word[:2])
    for l in word[2:]:
        if show_steps:
            h_before = h.copy()
        h[0] = c(o(h[0]) + o(h[1]))
        h[1] = c(o(h[1]) + o(l))
        if show_steps:
            print('| `{}` | {} | `{}` | {} | {} | {} | `{}` |'.format(
                  ''.join(h_before), 
                  (o(h_before[0]), o(h_before[1])),
                  l, 
                  o(l), 
                  o(h_before[0]) + o(h_before[1]), 
                  o(h_before[1]) + o(l), 
                  ''.join(h)))
    return ''.join(h)

In [9]:
whash1(sanitise('the cat'), show_steps=True)

| old code | code as<br>numbers | passphrase<br/>letter | number of<br/>letter | new first<br/>part of code |new second<br/>part of code | new code |
|:---|:---|:---|:---|:---|:---|:---|
| `th` | (20, 8) | `e` | 5 | 28 | 13 | `bm` |
| `bm` | (2, 13) | `c` | 3 | 15 | 16 | `op` |
| `op` | (15, 16) | `a` | 1 | 31 | 17 | `eq` |
| `eq` | (5, 17) | `t` | 20 | 22 | 37 | `vk` |


'vk'

In [10]:
passphrase = sanitise("the traveller in the grey riding-coat, who called himself mr. melville, was "
                      "contemplating the malice of which the gods are capable.")

In [24]:
whash1(passphrase)

'mc'

You arrive at your door but the passcode doesn't work!

Looking at the lock, you see it's a different model from last time you were here. Reluctantly, you return to the hotel front desk and join the queue for instructions. 

Indeed, the algorithm has been changed, "to improve security," they say.

The new algorithm starts with the same initial value of the code for every passphrase. In in this case, the initial code is `ri`. The algorithm also uses two contants, `alpha`= 5 and `beta` = 11.

Then, for each letter in the pass phrase, the code is updated as follows:

1. "Multiply" the second letter of the code by `alpha` then add the first letter of the code, replacing the first code letter
2. "Multiply" the current letter of the pass phrase by `beta` and add the second letter of the code, replacing the second letter of the code

"Multiplying" letters is done by converting the letters to their position in the alphabet (starting at one) and multiplying. For instance, to multiply `u` by 11, convert `u` to `21`, multiply by 11 (`21` × `11` = `231`), then convert back to a letter (`231` is larger than 26, so it becomes `23`, which is `w`).

Again, all letters are converted to lower-case and anything that isn't a letter is ignored.

For example, to find the code from the pass phrase `the cat`, the code starts as being the first two letters `ri`. When the first letter is encrypted, the first letter of the code becomes:

`r` + `i` × `alpha` = `18` + `9` × `5` = `63` = `k`

and the second letter of the code becomes:

`i` + `t` × `beta` = `9` + `20` × `11` = `229` = `u`

The passphrase letters are used to update the code as follows:

| old code | code as<br>numbers | passphrase<br/>letter | number of<br/>letter | new first<br/>part of code |new second<br/>part of code | new code |
|:---|:---|:---|:---|:---|:---|:---|
| `ri` | (18, 9) | `t` | 20 | 63 | 229 | `ku` |
| `ku` | (11, 21) | `h` | 8 | 116 | 109 | `le` |
| `le` | (12, 5) | `e` | 5 | 37 | 60 | `kh` |
| `kh` | (11, 8) | `c` | 3 | 51 | 41 | `yo` |
| `yo` | (25, 15) | `a` | 1 | 100 | 26 | `vz` |
| `vz` | (22, 26) | `t` | 20 | 152 | 246 | `vl` |


So the final code is `vl`

## Part 2
Your passphrase remains "the traveller in the grey riding-coat, who called himself mr. melville, was contemplating the malice of which the gods are capable."

Using this new algorithm, **what is your door code?**

# Worked example of solution: Part 2

This is almost identical to part 1, but the arithmetic is slightly different. Note the use of keyword arguments with default values, to allow the code to use different starting values.

In [12]:
o('u'), o('u') * 11, (o('u') * 11 - 1) % 26 + 1, c(o('u') * 11)

(21, 231, 23, 'w')

In [13]:
o('r'), o('i'), o('i') * 5, o('r') + o('i') * 5, c(o('r') + o('i') * 5)

(18, 9, 45, 63, 'k')

In [14]:
o('i'), o('t'), o('t') * 11, o('i') + o('t') * 11, c(o('i') + o('t') * 11)

(9, 20, 220, 229, 'u')

In [21]:
def whash2(word, h0=None, alpha=5, beta=11):
    if h0 is None:
        h = list('ri')
    else:
        h = list(h0)
    for l in word:
        h[0] = c(o(h[0]) + o(h[1]) * alpha)
        h[1] = c(o(h[1]) + o(l) * beta)
    return ''.join(h)

In [15]:
# Extended version that generates the tables used in the question text.
def whash2(word, h0=None, alpha=5, beta=11, show_steps=False):
    if show_steps:
        print('| old code | code as<br>numbers | passphrase<br/>letter | number of<br/>letter | new first<br/>part of code |'
              'new second<br/>part of code | new code |')
        print('|:---|:---|:---|:---|:---|:---|:---|')
    if h0 is None:
        h = list('ri')
    else:
        h = list(h0)
    for l in word:
        if show_steps:
            h_before = h.copy()
        h[0] = c(o(h[0]) + o(h[1]) * alpha)
        h[1] = c(o(h[1]) + o(l) * beta)
        if show_steps:
            print('| `{}` | {} | `{}` | {} | {} | {} | `{}` |'.format(
                  ''.join(h_before), 
                  (o(h_before[0]), o(h_before[1])),
                  l, 
                  o(l), 
                  o(h_before[0]) + o(h_before[1]) * alpha, 
                  o(h_before[1]) + o(l) * beta, 
                  ''.join(h)))
    return ''.join(h)

In [16]:
whash2(sanitise('the cat'), show_steps=True)

| old code | code as<br>numbers | passphrase<br/>letter | number of<br/>letter | new first<br/>part of code |new second<br/>part of code | new code |
|:---|:---|:---|:---|:---|:---|:---|
| `ri` | (18, 9) | `t` | 20 | 63 | 229 | `ku` |
| `ku` | (11, 21) | `h` | 8 | 116 | 109 | `le` |
| `le` | (12, 5) | `e` | 5 | 37 | 60 | `kh` |
| `kh` | (11, 8) | `c` | 3 | 51 | 41 | `yo` |
| `yo` | (25, 15) | `a` | 1 | 100 | 26 | `vz` |
| `vz` | (22, 26) | `t` | 20 | 152 | 246 | `vl` |


'vl'

In [22]:
whash2(passphrase)

'qb'